In [81]:
import re
import numpy as np
import pandas as pd
from nltk.stem import WordNetLemmatizer
import spacy 
nlp=spacy.load('en_core_web_lg')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize



from nltk.tokenize import word_tokenize

In [65]:
df = pd.read_csv(r'E:\NLP_200\Code Section 05\smsspamcollection.tsv', sep='\t')
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [6]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [ ]:
.value_counts()

In [82]:
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(df['message'])):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(df['message'][sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    #document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    #document = document.split()
    document = word_tokenize(document)
    
    
    stop_words = set(stopwords.words('english'))
    document = [w for w in document if not w in stop_words]
    
    
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

In [83]:
#df['documents']
pd.DataFrame(documents)
x= documents
y=df['label']

In [84]:
from sklearn.model_selection import train_test_split

#X = df['message']  
#y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print('Training Data Shape:' , X_train.shape)
print('Training Data Shape:' , X_test.shape)

Training Data Shape: (3733,)
Training Data Shape: (1839,)


In [85]:
y_train.value_counts()

ham     3232
spam     501
Name: label, dtype: int64

In [86]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect= CountVectorizer()

X_train_counts= count_vect.fit_transform(X_train)
X_train_counts.shape 

(3733, 7082)

In [87]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(3733, 7082)

In [88]:
pd.DataFrame(X_train_tfidf)[0]

0         (0, 7069)\t0.6019702680143677\n  (0, 4415)\t...
1         (0, 7048)\t0.06220835924135395\n  (0, 6330)\...
2         (0, 4959)\t0.5639269003584342\n  (0, 4583)\t...
3         (0, 6219)\t0.15567219572424026\n  (0, 4470)\...
4         (0, 7048)\t0.2119138905992704\n  (0, 6330)\t...
                              ...                        
3728      (0, 7048)\t0.10087994978401515\n  (0, 6928)\...
3729      (0, 5795)\t0.5474874445685368\n  (0, 3794)\t...
3730      (0, 5800)\t0.44622014318070863\n  (0, 5799)\...
3731      (0, 6345)\t0.4026376302519309\n  (0, 5520)\t...
3732      (0, 6119)\t0.4501915902498888\n  (0, 5763)\t...
Name: 0, Length: 3733, dtype: object

In [89]:
from sklearn.svm import LinearSVC 
clf =LinearSVC()
clf.fit(X_train_tfidf,y_train)

LinearSVC()

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

(3733, 7082)

In [91]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),])


text_clf.fit(X_train, y_train)  


predictions = text_clf.predict(X_test)

In [93]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[1586    7]
 [  12  234]]


In [77]:
print(metrics.classification_report(y_test,predictions)) 

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [94]:
print(metrics.accuracy_score(y_test,predictions))

0.989668297988037


In [33]:
text_clf.predict(['Ready to develop your Ai skills'])[0]

'ham'

In [34]:
text_clf.predict(['Ready to develop your Ai skills????????  '])[0]


'ham'

In [35]:
text_clf.predict(['Ready to develop your Ai skills @ibm !'])[0]

'ham'

In [37]:
text_clf.predict(['Congratulations , you won @ rolex'])[0]

array(['spam'], dtype=object)